In [ ]:
from neo4j import GraphDatabase

class Neo4jConnection:
    def __init__(self, uri, user, password, database):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
        self._database = database

    def close(self):
        self._driver.close()

    def query(self, query, parameters=None):
        with self._driver.session(database=self._database) as session:
            result = session.run(query, parameters)
            return [record for record in result]

# Usage
uri = "neo4j+s://neo4j.dev.everycure.org:7687"  # Replace with your Neo4j URI
user = "neo4j"  # Replace with your username
password = "changeme-123-safe!"  # Replace with your password
database = "everycure-v0.4.5"

conn = Neo4jConnection(uri, user, password, database)
result = conn.query("MATCH (n) RETURN n LIMIT 5")
print(result)
conn.close()

[<Record n=<Node element_id='4:066597ec-8503-4cd4-be72-7fc1cbf2acac:0' labels=frozenset({'biolink:NamedThing', 'Entity'}) properties={'name': 'lumen of tube', 'description': 'The inner space within a tube.', 'upstream_data_source': ['rtxkg2'], 'id': 'AEO:0000078', 'category': 'biolink:NamedThing'}>>, <Record n=<Node element_id='4:066597ec-8503-4cd4-be72-7fc1cbf2acac:1' labels=frozenset({'biolink:AnatomicalEntity', 'Entity'}) properties={'name': 'keratin-based structure', 'description': 'Acellular anatomical structure whose predominant constituent is keratin.', 'upstream_data_source': ['rtxkg2'], 'id': 'AEO:0000099', 'category': 'biolink:AnatomicalEntity'}>>, <Record n=<Node element_id='4:066597ec-8503-4cd4-be72-7fc1cbf2acac:2' labels=frozenset({'biolink:AnatomicalEntity', 'Entity'}) properties={'name': 'exoskeletal tissue', 'description': 'A component of the exoskeleton.', 'upstream_data_source': ['rtxkg2'], 'id': 'AEO:0000185', 'category': 'biolink:AnatomicalEntity'}>>, <Record n=<Nod

In [14]:
QUERY = """
MATCH (start {id: $node1}), (end {id: $node2})
MATCH p = allShortestPaths((start)-[*]-(end))
WITH p,
     [n IN nodes(p) | n.category] AS node_labels,
     [r IN relationships(p) | type(r)] AS rel_types
WITH [i IN range(0, size(rel_types)-1) | 
       node_labels[i] + ':' + rel_types[i]] + [node_labels[-1]] AS path_signature
RETURN path_signature, COUNT(*) AS count
ORDER BY count DESC
"""

# Provide your node IDs
node1 = "ATC:B06AA02"
node2 = "CHV:0000008136"

# Run the query via your connection object
results_raw = conn.query(QUERY, parameters={"node1": node1, "node2": node2})

# Transform results into desired JSON-friendly structure
simplified_results = [
    ({"path": record["path_signature"]}, record["count"]) for record in results_raw
]

# Optional: write to file
import json
with open("simplified_paths.json", "w") as f:
    json.dump(simplified_results, f, indent=2)

# Preview
print(simplified_results[:2])


/var/folders/vj/ks1_0k8x3t9ftrwcr0t9vjwr0000gn/T/ipykernel_81816/256043732.py:12: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self._driver.session(database=self._database) as session:


[({'path': ['biolink:Drug:close_match', 'biolink:ChemicalEntity:interacts_with', 'biolink:SmallMolecule:affects', 'biolink:Gene:gene_associated_with_condition', 'biolink:Disease:close_match', 'biolink:Disease']}, 26), ({'path': ['biolink:Drug:close_match', 'biolink:ChemicalEntity:interacts_with', 'biolink:Drug:related_to', 'biolink:Disease:subclass_of', 'biolink:Disease:close_match', 'biolink:Disease']}, 10)]
